In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import cv2
import os
import random
import shutil
import xml.etree.ElementTree as ET

In [ ]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 17088, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 17088 (delta 22), reused 5 (delta 5), pack-reused 17051 (from 4)
Receiving objects: 100% (17088/17088), 15.71 MiB | 16.27 MiB/s, done.
Resolving deltas: 100% (11723/11723), done.


In [ ]:
!cd yolov5 && pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.3/905.3 kB 15.4 MB/s eta 0:00:00


#loading yolov5

In [ ]:
model = torch.hub.load("ultralytics/yolov5", "yolov5s")

/usr/local/lib/python3.10/dist-packages/torch/hub.py:330: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


YOLOv5 🚀 2025-1-5 Python-3.10.12 torch-2.5.1+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 35.6MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


#Import Dataset from kaggle

In [ ]:
from google.colab import files
files.upload()
import os
os.makedirs("/root/.kaggle", exist_ok=True)
os.rename("kaggle.json", "/root/.kaggle/kaggle.json")
os.chmod("/root/.kaggle/kaggle.json", 600)

Saving kaggle.json to kaggle.json


In [ ]:
!kaggle datasets download -d andrewmvd/face-mask-detection
!unzip face-mask-detection.zip -d /content/dataset

Dataset URL: https://www.kaggle.com/datasets/andrewmvd/face-mask-detection
License(s): CC0-1.0
100% 398M/398M [00:15<00:00, 28.7MB/s]
100% 398M/398M [00:15<00:00, 26.4MB/s]
Archive:  face-mask-detection.zip
  inflating: /content/dataset/annotations/maksssksksss0.xml  
  inflating: /content/dataset/annotations/maksssksksss1.xml  
  inflating: /content/dataset/annotations/maksssksksss10.xml  
  inflating: /content/dataset/annotations/maksssksksss100.xml  
  inflating: /content/dataset/annotations/maksssksksss101.xml  
  inflating: /content/dataset/annotations/maksssksksss102.xml  
  inflating: /content/dataset/annotations/maksssksksss103.xml  
  inflating: /content/dataset/annotations/maksssksksss104.xml  
  inflating: /content/dataset/annotations/maksssksksss105.xml  
  inflating: /content/dataset/annotations/maksssksksss106.xml  
  inflating: /content/dataset/annotations/maksssksksss107.xml  
  inflating: /content/dataset/annotations/maksssksksss108.xml  
  inflating: /content/dataset/

Splitting dataset into train,test and val (%80,%10 and %10)and then converting the dataset to yaml for yolo

In [ ]:
images_dir = "/content/dataset/images"
annotations_dir = "/content/dataset/annotations"

# creating directories for yolo format
output_dir = "/content/dataset_yolo"
os.makedirs(f"{output_dir}/images/train", exist_ok=True)
os.makedirs(f"{output_dir}/images/val", exist_ok=True)
os.makedirs(f"{output_dir}/images/test", exist_ok=True)
os.makedirs(f"{output_dir}/labels/train", exist_ok=True)
os.makedirs(f"{output_dir}/labels/val", exist_ok=True)
os.makedirs(f"{output_dir}/labels/test", exist_ok=True)

In [ ]:
image_files = [f for f in os.listdir(images_dir) if f.endswith(".png")]

# shuffle and split all of the images
random.seed(42)
random.shuffle(image_files)
train_size = int(0.8 * len(image_files))
test_size=int(0.1 * len(image_files))
val_size = int(0.1 * len(image_files))
train_files = image_files[:train_size]
val_files = image_files[train_size:train_size + val_size]
test_files = image_files[train_size + val_size:]

Pascal VOC XML  to YOLO

In [ ]:
def convert_pascal_to_yolo(xml_file, image_width, image_height, classes):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    yolo_annotation = []

    for obj in root.findall("object"):
        class_name = obj.find("name").text
        if class_name not in classes:
            print(f"Class '{class_name}' not in defined classes. Skipping...")
            continue
        class_id = classes.index(class_name)
        bbox = obj.find("bndbox")
        try:
            xmin = int(bbox.find("xmin").text)
            ymin = int(bbox.find("ymin").text)
            xmax = int(bbox.find("xmax").text)
            ymax = int(bbox.find("ymax").text)
        except AttributeError as e:
            print(f"Bounding box missing in file: {xml_file}, skipping...")
            continue

        x_center = (xmin + xmax) / 2 / image_width
        y_center = (ymin + ymax) / 2 / image_height
        width = (xmax - xmin) / image_width
        height = (ymax - ymin) / image_height

        yolo_annotation.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

        if not yolo_annotation:
          print(f"No valid annotations found in {xml_file}")

        return "\n".join(yolo_annotation)


In [ ]:
classes = ['without_mask', 'with_mask', 'mask_weared_incorrect']

def process_files(image_files, split):
    for image_file in image_files:

        # paths for image and annotation
        image_path = os.path.join(images_dir, image_file)
        xml_file = os.path.join(annotations_dir, os.path.splitext(image_file)[0] + ".xml")

        # move image
        shutil.copy(image_path, f"{output_dir}/images/{split}/{image_file}")

        # convert it into yolo format
        tree = ET.parse(xml_file)
        root = tree.getroot()
        image_width = int(root.find("size/width").text)
        image_height = int(root.find("size/height").text)
        yolo_annotation = convert_pascal_to_yolo(xml_file, image_width, image_height, classes)

        # save yolo annotations
        txt_file = os.path.join(f"{output_dir}/labels/{split}", os.path.splitext(image_file)[0] + ".txt")
        with open(txt_file, "w") as f:
            f.write(yolo_annotation)

# process training, test and validation files
process_files(train_files, "train")
process_files(val_files, "val")
process_files(test_files, "test")


In [ ]:
# checking the dataset format and files
!ls -R /content/dataset_yolo


/content/dataset_yolo:
images	labels

/content/dataset_yolo/images:
test  train  val

/content/dataset_yolo/images/test:
maksssksksss111.png  maksssksksss284.png  maksssksksss470.png  maksssksksss663.png
maksssksksss123.png  maksssksksss286.png  maksssksksss472.png  maksssksksss670.png
maksssksksss136.png  maksssksksss288.png  maksssksksss494.png  maksssksksss673.png
maksssksksss140.png  maksssksksss293.png  maksssksksss4.png    maksssksksss678.png
maksssksksss148.png  maksssksksss351.png  maksssksksss501.png  maksssksksss680.png
maksssksksss155.png  maksssksksss368.png  maksssksksss513.png  maksssksksss681.png
maksssksksss157.png  maksssksksss369.png  maksssksksss533.png  maksssksksss694.png
maksssksksss165.png  maksssksksss383.png  maksssksksss536.png  maksssksksss6.png
maksssksksss171.png  maksssksksss385.png  maksssksksss544.png  maksssksksss700.png
maksssksksss182.png  maksssksksss386.png  maksssksksss546.png  maksssksksss715.png
maksssksksss192.png  maksssksksss388.png  maksssksk

Creating a YAML file

In [ ]:
# creating yaml configuration file
with open("/content/dataset_yolo/face_mask.yaml", "w") as f:
    f.write(f"""
    train: {output_dir}/images/train
    val: {output_dir}/images/val
    test: {output_dir}/images/test
    nc: {len(classes)}
    names: {classes}
    """)

In [ ]:
!cp "/content/dataset_yolo/face_mask.yaml" "/content/yolov5"

#Fine tuning YOLOv5

In [ ]:
!cd yolov5 && python train.py --img 640 --batch 16 --epochs 80 --data face_mask.yaml --weights yolov5s.pt  --workers 2 --freeze 20

wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-01-05 08:21:43.497613: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-05 08:21:43.540390: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-05 08:21:43.557557: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visua

#Evaluation

In [ ]:
!cd yolov5 && python val.py --task test --data face_mask.yaml --weights runs/train/exp/weights/best.pt


val: data=face_mask.yaml, weights=['runs/train/exp/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=test, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-392-gf003c3df Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
test: Scanning /content/dataset_yolo/labels/test.cache... 86 images, 0 backgrounds, 0 corrupt: 100% 86/86 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 3/3 [00:05<00:00,  1.89s/it]
                   all         86         86      0.436       0.46      0.407      0.287
          without_mask         86         14       0.44      0.394      0.449       0.34
             with_mask         86      

In [ ]:
model = torch.hub.load("ultralytics/yolov5", "custom", path="yolov5/runs/train/exp/weights/best.pt", force_reload=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2025-1-4 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)



Exception: [Errno 2] No such file or directory: 'yolov5/best.pt'. Cache may be out of date, try `force_reload=True` or see https://docs.ultralytics.com/yolov5/tutorials/pytorch_hub_model_loading for help.

#Real-time Detection with Opencv

Since Google colab does not sport webcam, i have downloaded the best weight file and do the real-time detection locally.